In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
#
# Copyright (c) 2022 Milan Ondrašovič <milan.ondrasovic@gmail.com>
#
# MIT License
#
# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE

from matplotlib import pyplot as plt
from PIL import Image

from compression import compress_image
from dct import extract_dct_blocks
from visual import plot_compression_effect

plt.style.use('dark_background')

%matplotlib inline

%load_ext autoreload
%autoreload 2

In [ ]:
image_orig = Image.open('uncompressed.tiff').convert('L')

image_compressed = compress_image(image_orig, quality=1)

dct_blocks = extract_dct_blocks(image_orig)
dct_blocks_compressed = extract_dct_blocks(image_compressed)

plt.imshow(image_orig, cmap='gray')

In [ ]:
plt.imshow(image_compressed, cmap='gray')

In [ ]:
fig = plot_compression_effect(
    [dct_blocks, dct_blocks_compressed], ['uncompressed', 'compressed']
)
fig.show()

In [ ]:
import numpy as np

# Luminance quantization coefficient.
luminance_qm = np.array(
    [[16,11,10,16,24,40,51,61],
     [12,12,14,19,26,48,60,55],
     [14,13,16,24,40,57,69,56],
     [14,17,22,29,51,87,80,62],
     [18,22,37,56,68,109,103,77],
     [24,35,55,64,81,104,113,92],
     [49,64,78,87,103,121,120,101],
     [72,92,95,98,112,100,103,99]], dtype=np.float32)
  
# Chromaticity quantization coefficient.
chromaticity_qm = np.array(
    [[17,18,24,47,99,99,99,99],
     [18,21,26,66,99,99,99,99],
     [24,26,56,99,99,99,99,99],
     [47,66,99,99,99,99,99,99],
     [99,99,99,99,99,99,99,99],
     [99,99,99,99,99,99,99,99],
     [99,99,99,99,99,99,99,99],
     [99,99,99,99,99,99,99,99]], dtype=np.float32)


image_compressed_new = compress_image(image_orig, quality=50)
curr_qm = image_compressed_new.quantization[0]

for curr_quant_val, standard_quant_val in zip(curr_qm, luminance_qm.flatten()):
    lower_scale = (100 * curr_quant_val - 150) / standard_quant_val
    upper_scale = (100 * curr_quant_val + 50) / standard_quant_val

    lower_quality_factor = int(round((200 - upper_scale) / 2))
    upper_quality_factor = int(round((200 - lower_scale) / 2))

    print(curr_quant_val, standard_quant_val, lower_quality_factor, upper_quality_factor)
